In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# MEMS Spring-Mass-Damper System Simulation\n",
    "\n",
    "**Author:** Silicon Fabrication Handbook  \n",
    "**License:** MIT  \n",
    "**Description:** Interactive simulation of a MEMS accelerometer modeled as a spring-mass-damper system\n",
    "\n",
    "---\n",
    "\n",
    "## Table of Contents\n",
    "\n",
    "1. [Introduction](#introduction)\n",
    "2. [Theory](#theory)\n",
    "3. [Setup and Imports](#setup)\n",
    "4. [MEMS Accelerometer Class](#class)\n",
    "5. [Step Response Analysis](#step-response)\n",
    "6. [Frequency Response (Bode Plot)](#frequency-response)\n",
    "7. [Noise Analysis](#noise-analysis)\n",
    "8. [Transient Response](#transient-response)\n",
    "9. [Design Space Exploration](#design-space)\n",
    "10. [Interactive Parameter Tuning](#interactive)\n",
    "11. [Summary and Conclusions](#summary)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Introduction <a id=\"introduction\"></a>\n",
    "\n",
    "MEMS (Micro-Electro-Mechanical Systems) accelerometers are widely used in:\n",
    "- Smartphones and tablets (screen rotation, step counting)\n",
    "- Automotive safety (airbag deployment, ESC)\n",
    "- Consumer electronics (gaming controllers, wearables)\n",
    "- Industrial applications (vibration monitoring)\n",
    "\n",
    "This notebook simulates a **capacitive MEMS accelerometer** using a **spring-mass-damper** model.\n",
    "\n",
    "### Key Features:\n",
    "- ✅ Time-domain response analysis\n",
    "- ✅ Frequency response (Bode plots)\n",
    "- ✅ Noise analysis (thermal + electronic)\n",
    "- ✅ Capacitive sensing simulation\n",
    "- ✅ Design trade-off exploration\n",
    "- ✅ Interactive parameter tuning"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Theory <a id=\"theory\"></a>\n",
    "\n",
    "### 2.1 Equation of Motion\n",
    "\n",
    "The proof mass motion is governed by:\n",
    "\n",
    "$$\n",
    "m\\ddot{x} + b\\dot{x} + kx = ma(t)\n",
    "$$\n",
    "\n",
    "Where:\n",
    "- $m$ = proof mass [kg]\n",
    "- $b$ = damping coefficient [N·s/m]\n",
    "- $k$ = spring constant [N/m]\n",
    "- $x$ = displacement [m]\n",
    "- $a(t)$ = input acceleration [m/s²]\n",
    "\n",
    "### 2.2 Key Parameters\n",
    "\n",
    "**Natural frequency:**\n",
    "$$\\omega_n = \\sqrt{\\frac{k}{m}} \\quad [rad/s]$$\n",
    "\n",
    "**Damping ratio:**\n",
    "$$\\zeta = \\frac{b}{2\\sqrt{mk}}$$\n",
    "\n",
    "**Quality factor:**\n",
    "$$Q = \\frac{1}{2\\zeta}$$\n",
    "\n",
    "### 2.3 Transfer Function\n",
    "\n",
    "In Laplace domain:\n",
    "$$H(s) = \\frac{X(s)}{A(s)} = \\frac{1}{s^2 + 2\\zeta\\omega_n s + \\omega_n^2}$$\n",
    "\n",
    "### 2.4 Capacitive Sensing\n",
    "\n",
    "Parallel plate capacitance:\n",
    "$$C = \\frac{\\epsilon_0 A}{d_0 - x}$$\n",
    "\n",
    "For small displacement:\n",
    "$$\\Delta C \\approx C_0 \\frac{x}{d_0}$$"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Setup and Imports <a id=\"setup\"></a>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Standard imports\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from scipy import signal\n",
    "from pathlib import Path\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# For interactive widgets\n",
    "try:\n",
    "    from ipywidgets import interact, FloatSlider, IntSlider, Output\n",
    "    import ipywidgets as widgets\n",
    "    WIDGETS_AVAILABLE = True\n",
    "except ImportError:\n",
    "    print(\"⚠️  ipywidgets not installed. Interactive features disabled.\")\n",
    "    print(\"   Install with: pip install ipywidgets\")\n",
    "    WIDGETS_AVAILABLE = False\n",
    "\n",
    "# Configure plotting\n",
    "%matplotlib inline\n",
    "plt.style.use('seaborn-v0_8-darkgrid')\n",
    "plt.rcParams['figure.figsize'] = (12, 6)\n",
    "plt.rcParams['font.size'] = 11\n",
    "plt.rcParams['lines.linewidth'] = 2\n",
    "\n",
    "# Create output directory\n",
    "OUTPUT_DIR = Path(\"images\")\n",
    "OUTPUT_DIR.mkdir(exist_ok=True)\n",
    "\n",
    "print(\"✓ Setup complete!\")\n",
    "print(f\"  NumPy version: {np.__version__}\")\n",
    "print(f\"  Output directory: {OUTPUT_DIR}\")\n",
    "print(f\"  Interactive widgets: {'Enabled' if WIDGETS_AVAILABLE else 'Disabled'}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. MEMS Accelerometer Class <a id=\"class\"></a>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class MEMSAccelerometer:\n",
    "    \"\"\"\n",
    "    MEMS accelerometer spring-mass-damper model.\n",
    "    \n",
    "    Parameters:\n",
    "        m: Proof mass [kg]\n",
    "        k: Spring constant [N/m]\n",
    "        b: Damping coefficient [N·s/m]\n",
    "        C0: Nominal capacitance [F]\n",
    "        d0: Nominal gap [m]\n",
    "        A: Electrode area [m²]\n",
    "    \"\"\"\n",
    "    \n",
    "    def __init__(self, m=1e-9, k=10, b=1e-6, C0=1e-12, d0=2e-6, A=1e-8):\n",
    "        self.m = m\n",
    "        self.k = k\n",
    "        self.b = b\n",
    "        self.C0 = C0\n",
    "        self.d0 = d0\n",
    "        self.A = A\n",
    "        \n",
    "        # Derived parameters\n",
    "        self.omega_n = np.sqrt(k / m)\n",
    "        self.f_n = self.omega_n / (2 * np.pi)\n",
    "        self.zeta = b / (2 * np.sqrt(m * k))\n",
    "        self.Q = 1 / (2 * self.zeta) if self.zeta > 0 else np.inf\n",
    "        self.sensitivity = 1 / (k / m)\n",
    "    \n",
    "    def __repr__(self):\n",
    "        return f\"\"\"MEMS Accelerometer:\n",
    "  Mass: {self.m*1e9:.2f} ng\n",
    "  Spring constant: {self.k:.2f} N/m\n",
    "  Damping: {self.b*1e6:.3f} µN·s/m\n",
    "  Natural frequency: {self.f_n/1e3:.2f} kHz\n",
    "  Damping ratio ζ: {self.zeta:.3f}\n",
    "  Quality factor Q: {self.Q:.1f}\n",
    "  Sensitivity: {self.sensitivity*1e9:.2f} nm/g\"\"\"\n",
    "    \n",
    "    def transfer_function(self):\n",
    "        \"\"\"Return H(s) = X(s)/A(s)\"\"\"\n",
    "        num = [1]\n",
    "        den = [1, 2*self.zeta*self.omega_n, self.omega_n**2]\n",
    "        return signal.TransferFunction(num, den)\n",
    "    \n",
    "    def step_response(self, acceleration=9.81, t_max=0.01, n_points=1000):\n",
    "        \"\"\"Simulate step response.\"\"\"\n",
    "        sys = self.transfer_function()\n",
    "        t = np.linspace(0, t_max, n_points)\n",
    "        t_step, x = signal.step(sys, T=t)\n",
    "        return t_step, x * acceleration\n",
    "    \n",
    "    def frequency_response(self, f_min=1, f_max=1e6, n_points=1000):\n",
    "        \"\"\"Calculate frequency response.\"\"\"\n",
    "        sys = self.transfer_function()\n",
    "        w = 2 * np.pi * np.logspace(np.log10(f_min), np.log10(f_max), n_points)\n",
    "        w, H = signal.freqs(sys.num, sys.den, worN=w)\n",
    "        return w / (2 * np.pi), np.abs(H)\n",
    "    \n",
    "    def displacement_to_capacitance(self, x):\n",
    "        \"\"\"Convert displacement to capacitance.\"\"\"\n",
    "        epsilon_0 = 8.854e-12\n",
    "        return epsilon_0 * self.A / (self.d0 - x)\n",
    "    \n",
    "    def capacitance_to_voltage(self, C, V_bias=1.0, C_f=1e-12):\n",
    "        \"\"\"Convert capacitance to voltage.\"\"\"\n",
    "        delta_C = C - self.C0\n",
    "        return -V_bias * delta_C / C_f\n",
    "\n",
    "# Create default instance\n",
    "accel = MEMSAccelerometer()\n",
    "print(accel)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Step Response Analysis <a id=\"step-response\"></a>\n",
    "\n",
    "Analyze the response to a **1g constant acceleration** (step input)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Simulate step response\n",
    "t, x = accel.step_response(acceleration=9.81, t_max=0.01)\n",
    "x_nm = x * 1e9  # Convert to nanometers\n",
    "\n",
    "# Calculate settling time (2% criterion)\n",
    "steady_state = x_nm[-1]\n",
    "settling_idx = np.where(np.abs(x_nm - steady_state) < 0.02 * steady_state)[0][0]\n",
    "settling_time = t[settling_idx]\n",
    "\n",
    "# Plot\n",
    "fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))\n",
    "\n",
    "# Displacement\n",
    "ax1.plot(t * 1e3, x_nm, 'b-', linewidth=2.5, label='Displacement')\n",
    "ax1.axhline(steady_state, color='r', linestyle='--', linewidth=1.5,\n",
    "            label=f'Steady state: {steady_state:.2f} nm')\n",
    "ax1.axvline(settling_time * 1e3, color='g', linestyle='--', linewidth=1.5,\n",
    "            label=f'Settling time: {settling_time*1e3:.2f} ms')\n",
    "ax1.set_xlabel('Time [ms]', fontsize=12)\n",
    "ax1.set_ylabel('Displacement [nm]', fontsize=12)\n",
    "ax1.set_title('Step Response to 1g Acceleration', fontsize=14, fontweight='bold')\n",
    "ax1.legend(fontsize=10, loc='lower right')\n",
    "ax1.grid(True, alpha=0.3)\n",
    "\n",
    "# Capacitance change\n",
    "C = accel.displacement_to_capacitance(x)\n",
    "delta_C_fF = (C - accel.C0) * 1e15\n",
    "ax2.plot(t * 1e3, delta_C_fF, 'r-', linewidth=2.5)\n",
    "ax2.set_xlabel('Time [ms]', fontsize=12)\n",
    "ax2.set_ylabel('Capacitance Change [fF]', fontsize=12)\n",
    "ax2.set_title('Capacitive Sensor Response', fontsize=14, fontweight='bold')\n",
    "ax2.grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig(OUTPUT_DIR / 'step_response.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "print(f\"📊 Results:\")\n",
    "print(f\"  • Steady-state displacement: {steady_state:.2f} nm\")\n",
    "print(f\"  • Settling time (2%): {settling_time*1e3:.2f} ms\")\n",
    "print(f\"  • Peak capacitance change: {np.max(delta_C_fF):.2f} fF\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Frequency Response (Bode Plot) <a id=\"frequency-response\"></a>\n",
    "\n",
    "Analyze how the accelerometer responds to different input frequencies."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate frequency response\n",
    "f, H = accel.frequency_response(f_min=10, f_max=1e6, n_points=1000)\n",
    "\n",
    "# Convert to dB and phase\n",
    "mag_dB = 20 * np.log10(H)\n",
    "phase = np.angle(H * 2 * np.pi * f, deg=True)\n",
    "\n",
    "# Find -3dB bandwidth\n",
    "mag_max = np.max(mag_dB)\n",
    "bw_idx = np.where(mag_dB >= mag_max - 3)[0]\n",
    "bandwidth = f[bw_idx[-1]] if len(bw_idx) > 0 else f[-1]\n",
    "\n",
    "# Plot Bode diagram\n",
    "fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))\n",
    "\n",
    "# Magnitude\n",
    "ax1.semilogx(f, mag_dB, 'b-', linewidth=2.5)\n",
    "ax1.axhline(mag_max - 3, color='r', linestyle='--', linewidth=1.5, label='-3dB line')\n",
    "ax1.axvline(bandwidth, color='g', linestyle='--', linewidth=1.5,\n",
    "            label=f'BW: {bandwidth/1e3:.1f} kHz')\n",
    "ax1.axvline(accel.f_n, color='orange', linestyle=':', linewidth=2,\n",
    "            label=f'$f_n$: {accel.f_n/1e3:.1f} kHz')\n",
    "ax1.set_ylabel('Magnitude [dB]', fontsize=12)\n",
    "ax1.set_title('Frequency Response (Bode Plot)', fontsize=14, fontweight='bold')\n",
    "ax1.legend(fontsize=10)\n",
    "ax1.grid(True, alpha=0.3, which='both')\n",
    "\n",
    "# Phase\n",
    "ax2.semilogx(f, phase, 'r-', linewidth=2.5)\n",
    "ax2.set_xlabel('Frequency [Hz]', fontsize=12)\n",
    "ax2.set_ylabel('Phase [degrees]', fontsize=12)\n",
    "ax2.grid(True, alpha=0.3, which='both')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig(OUTPUT_DIR / 'frequency_response.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "print(f\"📊 Results:\")\n",
    "print(f\"  • -3dB Bandwidth: {bandwidth/1e3:.1f} kHz\")\n",
    "print(f\"  • Resonant peak: {mag_max:.1f} dB\")\n",
    "print(f\"  • Quality factor: {accel.Q:.1f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Noise Analysis <a id=\"noise-analysis\"></a>\n",
    "\n",
    "Analyze **thermal (Brownian) noise** and **electronic noise** sources."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Frequency range\n",
    "f_noise = np.logspace(0, 6, 1000)  # 1 Hz to 1 MHz\n",
    "\n",
    "# Constants\n",
    "k_B = 1.38e-23  # Boltzmann constant\n",
    "T = 300  # Temperature [K]\n",
    "\n",
    "# Thermal (Brownian) noise\n",
    "S_x_thermal = np.sqrt(4 * k_B * T * accel.b / accel.k**2)\n",
    "x_thermal = S_x_thermal * np.ones_like(f_noise)\n",
    "a_thermal = x_thermal * accel.k / accel.m / 9.81  # Convert to g/√Hz\n",
    "\n",
    "# Electronic noise (charge amplifier)\n",
    "R_f = 1e9  # Feedback resistor [Ω]\n",
    "C_f = 1e-12  # Feedback capacitor [F]\n",
    "v_n_amplifier = np.sqrt(4 * k_B * T * R_f)\n",
    "\n",
    "V_bias = 1.0\n",
    "sens_V_per_g = V_bias * accel.C0 / C_f / accel.d0 * accel.sensitivity\n",
    "a_electronic = v_n_amplifier / sens_V_per_g / 9.81 * np.ones_like(f_noise)\n",
    "\n",
    "# Total noise\n",
    "a_total = np.sqrt(a_thermal**2 + a_electronic**2)\n",
    "\n",
    "# Integrate over bandwidth\n",
    "bandwidth = 100  # Hz\n",
    "f_bw = f_noise[f_noise <= bandwidth]\n",
    "a_total_bw = a_total[f_noise <= bandwidth]\n",
    "noise_rms = np.sqrt(np.trapz(a_total_bw**2, f_bw))\n",
    "\n",
    "# Plot\n",
    "fig, ax = plt.subplots(figsize=(12, 6))\n",
    "\n",
    "ax.loglog(f_noise, a_thermal * 1e6, 'b-', linewidth=2.5, label='Thermal (Brownian) noise')\n",
    "ax.loglog(f_noise, a_electronic * 1e6, 'r-', linewidth=2.5, label='Electronic noise')\n",
    "ax.loglog(f_noise, a_total * 1e6, 'k-', linewidth=2.5, label='Total noise')\n",
    "ax.axvline(bandwidth, color='g', linestyle='--', linewidth=1.5,\n",
    "           label=f'Bandwidth: {bandwidth} Hz')\n",
    "\n",
    "ax.set_xlabel('Frequency [Hz]', fontsize=12)\n",
    "ax.set_ylabel('Noise Spectral Density [µg/√Hz]', fontsize=12)\n",
    "ax.set_title('Noise Analysis', fontsize=14, fontweight='bold')\n",
    "ax.legend(fontsize=10)\n",
    "ax.grid(True, alpha=0.3, which='both')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig(OUTPUT_DIR / 'noise_analysis.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "print(f\"📊 Results:\")\n",
    "print(f\"  • Thermal noise: {a_thermal[0]*1e6:.2f} µg/√Hz\")\n",
    "print(f\"  • Electronic noise: {a_electronic[0]*1e6:.2f} µg/√Hz\")\n",
    "print(f\"  • Total noise: {a_total[0]*1e6:.2f} µg/√Hz\")\n",
    "print(f\"  • RMS noise (0-{bandwidth} Hz): {noise_rms*1e6:.2f} µg\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Transient Response <a id=\"transient-response\"></a>\n",
    "\n",
    "Simulate response to a **time-varying sinusoidal acceleration**."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Time vector\n",
    "t_trans = np.linspace(0, 0.02, 2000)  # 20 ms\n",
    "\n",
    "# Input: 1g sine wave at 500 Hz\n",
    "f_input = 500  # Hz\n",
    "a_input = 9.81 * np.sin(2 * np.pi * f_input * t_trans)\n",
    "\n",
    "# Simulate\n",
    "sys = accel.transfer_function()\n",
    "t_sim, x_trans, _ = signal.lsim(sys, a_input, t_trans)\n",
    "\n",
    "# Convert to capacitance and voltage\n",
    "C_trans = accel.displacement_to_capacitance(x_trans)\n",
    "V_out = accel.capacitance_to_voltage(C_trans)\n",
    "\n",
    "# Plot\n",
    "fig, axes = plt.subplots(3, 1, figsize=(12, 10))\n",
    "\n",
    "# Input acceleration\n",
    "axes[0].plot(t_sim * 1e3, a_input / 9.81, 'b-', linewidth=2)\n",
    "axes[0].set_ylabel('Acceleration [g]', fontsize=12)\n",
    "axes[0].set_title('Input: 1g Sine Wave at 500 Hz', fontsize=14, fontweight='bold')\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "\n",
    "# Displacement\n",
    "axes[1].plot(t_sim * 1e3, x_trans * 1e9, 'r-', linewidth=2)\n",
    "axes[1].set_ylabel('Displacement [nm]', fontsize=12)\n",
    "axes[1].set_title('Proof Mass Displacement', fontsize=14, fontweight='bold')\n",
    "axes[1].grid(True, alpha=0.3)\n",
    "\n",
    "# Output voltage\n",
    "axes[2].plot(t_sim * 1e3, V_out * 1e3, 'g-', linewidth=2)\n",
    "axes[2].set_xlabel('Time [ms]', fontsize=12)\n",
    "axes[2].set_ylabel('Output Voltage [mV]', fontsize=12)\n",
    "axes[2].set_title('Capacitive Readout Signal', fontsize=14, fontweight='bold')\n",
    "axes[2].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig(OUTPUT_DIR / 'transient_response.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "print(f\"📊 Results:\")\n",
    "print(f\"  • Peak displacement: {np.max(np.abs(x_trans))*1e9:.2f} nm\")\n",
    "print(f\"  • Peak output voltage: {np.max(np.abs(V_out))*1e3:.2f} mV\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Design Space Exploration <a id=\"design-space\"></a>\n",
    "\n",
    "Explore trade-offs between **mass**, **spring constant**, and performance metrics."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Parameter ranges\n",
    "masses = np.logspace(-10, -8, 50)  # 0.1 ng to 10 ng\n",
    "spring_constants = np.logspace(0, 2, 50)  # 1 to 100 N/m\n",
    "\n",
    "M, K = np.meshgrid(masses, spring_constants)\n",
    "\n",
    "# Calculate metrics\n",
    "f_n_map = np.sqrt(K / M) / (2 * np.pi)  # Natural frequency [Hz]\n",
    "sensitivity_map = M / K * 9.81 * 1e9  # Sensitivity [nm/g]\n",
    "\n",
    "# Thermal noise (simplified)\n",
    "b_fixed = 1e-6\n",
    "noise_map = np.sqrt(4 * k_B * T * b_fixed / K**2) * K / M / 9.81 * 1e6  # µg/√Hz\n",
    "\n",
    "# Plot\n",
    "fig, axes = plt.subplots(1, 3, figsize=(16, 4.5))\n",
    "\n",
    "# Natural frequency\n",
    "c1 = axes[0].contourf(M * 1e9, K, f_n_map / 1e3, levels=20, cmap='viridis')\n",
    "axes[0].set_xlabel('Mass [ng]', fontsize=11)\n",
    "axes[0].set_ylabel('Spring Constant [N/m]', fontsize=11)\n",
    "axes[0].set_title('Natural Frequency [kHz]', fontsize=13, fontweight='bold')\n",
    "axes[0].set_xscale('log')\n",
    "axes[0].set_yscale('log')\n",
    "plt.colorbar(c1, ax=axes[0])\n",
    "\n",
   # Continue from Design Space Exploration - Cell completion

# Sensitivity
c2 = axes[1].contourf(M * 1e9, K, sensitivity_map, levels=20, cmap='plasma')
axes[1].set_xlabel('Mass [ng]', fontsize=11)
axes[1].set_ylabel('Spring Constant [N/m]', fontsize=11)
axes[1].set_title('Sensitivity [nm/g]', fontsize=13, fontweight='bold')
axes[1].set_xscale('log')
axes[1].set_yscale('log')
plt.colorbar(c2, ax=axes[1])

# Noise floor
c3 = axes[2].contourf(M * 1e9, K, noise_map, levels=20, cmap='coolwarm')
axes[2].set_xlabel('Mass [ng]', fontsize=11)
axes[2].set_ylabel('Spring Constant [N/m]', fontsize=11)
axes[2].set_title('Noise Floor [µg/√Hz]', fontsize=13, fontweight='bold')
axes[2].set_xscale('log')
axes[2].set_yscale('log')
plt.colorbar(c3, ax=axes[2])

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'design_space.png', dpi=300, bbox_inches='tight')
plt.show()

print("📊 Design Trade-offs:")
print("  • High mass → Better sensitivity, lower noise")
print("  • High k → Higher bandwidth, lower sensitivity")
print("  • Optimal design depends on application requirements")


# ============================================================================
# Section 10: Interactive Parameter Tuning
# ============================================================================

print("\n" + "="*60)
print("SECTION 10: INTERACTIVE PARAMETER TUNING")
print("="*60)

if WIDGETS_AVAILABLE:
    output = Output()
    
    def update_accelerometer(mass_ng, k, zeta):
        """Interactive widget callback"""
        with output:
            output.clear_output(wait=True)
            
            # Create accelerometer with new parameters
            m = mass_ng * 1e-9
            b = 2 * zeta * np.sqrt(m * k)
            
            accel_interactive = MEMSAccelerometer(m=m, k=k, b=b)
            
            # Calculate responses
            t, x = accel_interactive.step_response(t_max=0.01)
            f, H = accel_interactive.frequency_response(f_min=10, f_max=1e6)
            
            # Create figure
            fig, axes = plt.subplots(1, 2, figsize=(14, 5))
            
            # Step response
            axes[0].plot(t * 1e3, x * 1e9, 'b-', linewidth=2.5)
            axes[0].set_xlabel('Time [ms]', fontsize=11)
            axes[0].set_ylabel('Displacement [nm]', fontsize=11)
            axes[0].set_title('Step Response (1g)', fontsize=12, fontweight='bold')
            axes[0].grid(True, alpha=0.3)
            
            # Frequency response
            axes[1].loglog(f, H, 'r-', linewidth=2.5)
            axes[1].axvline(accel_interactive.f_n, color='g', linestyle='--', 
                           label=f'fn = {accel_interactive.f_n/1e3:.1f} kHz')
            axes[1].set_xlabel('Frequency [Hz]', fontsize=11)
            axes[1].set_ylabel('Magnitude', fontsize=11)
            axes[1].set_title('Frequency Response', fontsize=12, fontweight='bold')
            axes[1].legend(fontsize=10)
            axes[1].grid(True, alpha=0.3, which='both')
            
            plt.tight_layout()
            plt.show()
            
            # Print parameters
            print(f"\n📐 Current Parameters:")
            print(f"  • Mass: {mass_ng:.2f} ng")
            print(f"  • Spring constant: {k:.2f} N/m")
            print(f"  • Damping ratio ζ: {zeta:.3f}")
            print(f"  • Natural frequency: {accel_interactive.f_n/1e3:.2f} kHz")
            print(f"  • Quality factor Q: {accel_interactive.Q:.1f}")
            print(f"  • Sensitivity: {accel_interactive.sensitivity*1e9:.2f} nm/g")
    
    # Create interactive widgets
    interact(update_accelerometer,
             mass_ng=FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0, 
                                description='Mass [ng]'),
             k=FloatSlider(min=1, max=100, step=1, value=10, 
                          description='k [N/m]'),
             zeta=FloatSlider(min=0.01, max=2.0, step=0.01, value=0.5, 
                             description='ζ'))
    
    display(output)
    
else:
    print("\n⚠️  Interactive widgets not available.")
    print("Install ipywidgets to enable: pip install ipywidgets")
    print("Then restart the kernel and re-run this notebook.")


# ============================================================================
# Section 11: Summary and Conclusions
# ============================================================================

print("\n" + "="*60)
print("SECTION 11: SUMMARY AND CONCLUSIONS")
print("="*60)

print("""
📋 MEMS Accelerometer Design Summary
=====================================

Key Findings:
-------------
1. **Natural Frequency**: Determines bandwidth and response speed
   • Higher fn → Faster response, wider bandwidth
   • Lower fn → Better low-frequency sensitivity

2. **Damping Ratio (ζ)**:
   • ζ < 0.7: Underdamped (oscillations, overshoot)
   • ζ ≈ 0.7: Critically damped (optimal transient response)
   • ζ > 1.0: Overdamped (slow, no overshoot)

3. **Quality Factor (Q)**:
   • High Q: Sharp resonance, low damping
   • Low Q: Broad response, high damping
   • Trade-off: Q vs. bandwidth

4. **Noise Sources**:
   • Thermal (Brownian) noise: Fundamental limit
   • Electronic noise: From readout circuit
   • Dominant noise depends on design parameters

5. **Design Trade-offs**:
   • Sensitivity ↔ Bandwidth
   • Noise ↔ Size (mass)
   • Dynamic range ↔ Gap size

Design Guidelines:
------------------
• For high-bandwidth applications (airbags): 
  → High k, low m, ζ ≈ 0.7

• For high-sensitivity applications (seismology):
  → Low k, high m, low ζ

• For general purpose (consumer electronics):
  → Balanced design with ζ ≈ 0.5-0.7

Typical MEMS Accelerometer Specs:
----------------------------------
• Mass: 0.1 - 10 ng
• Spring constant: 1 - 100 N/m
• Natural frequency: 1 - 100 kHz
• Sensitivity: 10 - 1000 nm/g
• Noise floor: 1 - 100 µg/√Hz
• Dynamic range: ±2g to ±200g

Next Steps:
-----------
1. Finite Element Analysis (FEA) for detailed mechanical design
2. Process simulation for fabrication planning
3. Circuit design for capacitive readout
4. Packaging considerations (stress isolation)
5. Temperature compensation strategies
6. Calibration and testing procedures

Files Generated:
----------------
""")

# List all generated images
image_files = list(OUTPUT_DIR.glob('*.png'))
if image_files:
    print("Generated plots:")
    for img_file in sorted(image_files):
        print(f"  • {img_file.name}")
else:
    print("  (No images found - run all cells to generate plots)")

print(f"""
Total images saved: {len(image_files)}
Output directory: {OUTPUT_DIR.absolute()}

=====================================
✅ Analysis Complete!
=====================================

Thank you for using the MEMS Spring-Mass-Damper Simulator!

For questions or issues, please refer to:
• Silicon Fabrication Handbook GitHub repository
• MEMS design textbooks (Senturia, Maluf, etc.)
• Online MEMS communities and forums

Happy designing! 🔬
""")